In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt 
import numpy as np
from functools import partial
import json
from glob import glob

import cv2
import os

#import nia22
#from nia22.generator import *

In [50]:
pwd

'/home/hoseung/Work/NIA/nia/scripts'

In [3]:
import csv

In [6]:
csv.reader?

In [30]:
cr = csv.reader(open("../test.csv"))

In [14]:
ll = [1,2,3,4]

In [16]:
ll

[1, 2, 3, 4]

In [17]:
cr

In [15]:
for l in ll:
    print(l)

1
2
3
4


In [31]:
for ll in cr:
    print(ll)

['성명', '피시험자 ID', '의료검사코드', '나이', '안경착용', '마스크착용', '시나리오 수', '촬영날짜', '비고']
['김태훈', '1', 'OB001', '27세 (1996년생)', '미착용', '미착용', '453개', '2022-08-02', '']
['윤지환', '2', 'OB002', '27세 (1996년생)', '착용', '미착용', '', '2022-08-02', '']
['김은서', '3', 'OB003', '20세 (2003년생)', '착용', '미착용', '', '2022-08-03', '']
['천정호', '4', 'OB004', '56세 (1967년생)', '미착용', '미착용', '', '2022-08-03', '']
['이경희', '5', 'OB005', '50세 (1973년생)', '착용', '미착용', '', '2022-08-04', '']
['이수미', '6', 'OB006', '47세 (1976년생) ', '미착용', '미착용', '', '2022-08-04', '']
['한민선', '7', 'OB007', '25세 (1998년생) ', '착용', '미착용', '', '2022-08-04', '']
['정혜은', '8', 'OB008', '46세 (1977년생)', '미착용', '미착용', '', '2022-08-05', '']
['오은숙', '9', 'OB009', '30세 (1993년생) ', '미착용', '미착용', '', '2022-08-08', '']
['손현정', '10', 'OB0010', '47세 (1976년생)', '착용', '미착용', '', '2022-08-08', '']
['정철호', '11', 'OB0011', '59세 (1964년생)', '착용', '미착용', '', '2022-08-09', '']
['심정곤', '12', 'OB0012', '54세 (1969년생) ', '착용', '미착용', '', '2022-08-09', '']
['임영준', '13', 'OB0013', '24세(1

In [32]:
ll

['임영준', '13', 'OB0013', '24세(1999년생)', '미착용', '미착용', '', '2022-08-10', '']

In [34]:
new_id = ll[1]

In [35]:
new_id

'13'

In [37]:
ss = ll[3]

In [45]:
ss

'24세(1999년생)'

In [47]:
if ss[0] =='1':
    age_string = "10대"
elif ss[0] =='2':
    age_string = "20대"    
elif ss[0] =='3':
    age_string = "30대"
elif ss[0] =='4':
    age_string== "30대"
elif ss[0] =='5':
    age_string = "30대"
elif ss[0] =='6':
    age_string = "30대"    
else:
    print("뭔가이상함")
print(age_string)

20대


In [4]:
def get_converter(field:str):
    if field == "inst":
        keys = ["S", "T", "L", "V", "M"]
    elif field == "condition":
        keys = ["F", "S", "D", "A", "N"]
    elif field == "posture":
        keys = ["S", "D", "P", "L", "F", 
                "C", "H", "E", "T", "U"]
    else:
        raise ValueError(field)

    return partial(_concat, keys=keys)

def _concat(value, keys=None):
    """concatenate strings"""
    code =""
    for s in keys:
        if s in value:
            code += s
        else:
            code += "X"
    return code

# get total file size
def get_total_file_size(wdir, pattern):
    all_files = glob(wdir+ pattern)
    nfiles = len(all_files)
    
    print(f"found {nfiles} files")
    
    file_sizes_Bytes = [os.path.getsize(cc) for cc in all_files]
    tsize_inMB = np.sum(file_sizes_Bytes) / 1024 / 1024
    if tsize_inMB > 1024:
        return nfiles, f"{tsize_inMB/1024:.2f}GB"
    else:
        return nfiles, f"{tsize_inMB:.2f}MB"

In [5]:
concat_inst = get_converter("inst")
concat_condition = get_converter("condition")
concat_posture = get_converter("posture")

In [6]:
clip_dir = "/home/hoseung/Dropbox/DeepInsight/2022/NIA/1Cycle/FHD/done/"
clip_list = glob(clip_dir+"*.mp4")
clip_list.sort()
clip_list_fn = [cl.split("/")[-1] for cl in clip_list]

Eye-tracker, 얼굴 지향각, 시선 각, 카메라-촬영자 거리, 카메라-디스플레이 거리   
각각은 영상의 매 프레임마다 측정값이 제공 됨.  

이번에는 가짜 값을 만들어냄.

In [7]:
def point_to_degree(xpos, ypos, d_cam, d_disp, ):
    """transformation이 너무... 많은데..?? 
    """

In [9]:
# count the number of frames of the video
conditions = ["F", "S", "D", "A", "N"]
postures = ["S", "D", "P", "L", "F", "C", "H", "E", "T", "U"]
display = ["S", "T", "L", "V", "M"]

wdir = "./"

for fn_clip in clip_list:
    file_name = fn_clip.split("/")[-1]
    display, posture, condition, orientation = file_name.split(".mp4")[0].split("_")
    cap = cv2.VideoCapture(fn_clip)
    if not cap.isOpened():
        continue
    nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    #print(display, posture, condition, orientation)
    fake = Fake_generator(wdir, nframes, 1920, 1080, display, posture, condition, orientation)
    dist = 35 # 정하고 하는거 아님
    fake.gen_d_disp(dist)
    fake.gen_d_cam(dist)
    fake.gen_head()
    fake.gen_pose_d()
    fake.gen_gaze_eye_trakcer()
    fake.save_all()

# Save meta data

In [19]:
now = str(datetime.now())
date, hour = now.split(" ")
hour = hour[:-3]

## Variables below are to be read from csv file ##
username = "hoseung"
email = "hschoi@dinsight.ai"
subject = 1
medical = "ABC1"
age = 34
glasses = "no"
cosmetic= "no"
mask = "no"
ver  = 2
rev = str(0)
loc = "S101"
inst = "STLVM"
condition = "FSDAN"
posture = "SDPLFCHETU"

n_point, tsize_point = get_total_file_size(wdir, "*point.csv")
n_head, tsize_head = get_total_file_size(wdir, "*head.csv")
n_gaze, tsize_gaze = get_total_file_size(wdir, "*gaze.csv")
n_cam, tsize_cam = get_total_file_size(wdir, "*[V12]_cam.csv") 
n_dcam, tsize_d_cam = get_total_file_size(wdir, "*d_cam.csv") # was display pose
n_disp, tsize_d_disp = get_total_file_size(wdir, "*d_disp.csv")
n_rgb, tsize_rgb = get_total_file_size(clip_dir, "*mp4")
n_ir, tsize_ir = get_total_file_size(wdir, "*ir*.mp4")
######################################

x = {
    "id": {"subject":subject, 
           "medical":medical},
    "age":age, 
    "glasses":glasses, 
    "cosmetic":cosmetic, 
    "mask":mask, 
    "ver":ver, 
    "date":date, 
    "loc":loc, 
    "inst":concat_inst(inst), 
    "condition":concat_condition(condition),
    "posture":concat_posture(posture),
    "rev":rev,
    "image": {"format": {"rgb": "mp4","ir": "mp4", "point":"csv", "head":"csv", "gaze":"csv", "cam":"csv",
                         "d_cam":"csv", "d_disp":"csv"},
                "spec":{"rgb.w":1920, "rgb.h":1080, "ir.w":1920, "ir.h":1080, 
                         "point":"yyyy-mm-dd,hh:mm:ss,x,y",
                         "head":"yyyy-mm-dd,hh:mm:ss,deg,deg,deg",
                         "gaze":"yyyy-mm-dd,hh:mm:ss,deg,deg,deg",
                         "cam":"yyyy-mm-dd,hh:mm:ss,deg,deg,deg",
                         "d_cam":"yyyy-mm-dd,hh:mm:ss,cm",
                         "d_disp":"yyyy-mm-dd,hh:mm:ss,cm"},
                "total_size":{"rgb":tsize_rgb, "ir":tsize_ir, "point":tsize_point, "head":tsize_head,
                              "gaze":tsize_gaze, "cam":tsize_cam, "d_cam":tsize_d_cam, "d_disp":tsize_d_disp},
                "total_count":{"rgb":n_rgb, "ir":n_ir, "point":n_point, "head":n_head, "gaze":n_gaze,
                               "cam":n_cam, "d_cam":n_dcam, "d_disp":n_disp}
             },
    "task":{
        "name":"NIA_EYE",
        "created": date + ", " + hour, 
        "updated":"", 
        "username":username, 
        "email":email},
    "filelist":clip_list_fn
}

# Dump json file
fn = f"{subject:03d}.json"
with open(fn, "w") as f:
    json.dump(x, f, indent=2)

found 104 files
found 104 files
found 104 files
found 104 files
found 104 files
found 104 files
found 104 files
found 0 files


In [ ]:
1`